In [2]:
%matplotlib inline
import sklearn.cluster as cluster
import matplotlib.pyplot as plt
import numpy as np
import re
import pandas as pd
import math
from sklearn.neighbors import KNeighborsClassifier as KNNCLF
from sklearn.preprocessing import Normalizer as normalizer
from sklearn.cross_validation import cross_val_score as cross_val
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.neural_network import MLPClassifier
from scipy import stats

/Users/Martin/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [3]:
Crimes = pd.read_csv('crime.csv')
Crimes = Crimes[np.isfinite(Crimes['Police_Districts'])]
Crimes = Crimes[np.isfinite(Crimes['UCR_General'])]
CrimesData = Crimes.as_matrix()
datainit = CrimesData[:,[10,2,5,8,12,13]]

In [4]:
data = datainit

In [21]:
datala = datainit[:,3].tolist()
counts = {}
countspr = {}
for crime in range(100,2601,100):
    counts[crime]=datala.count(crime)
    countspr[crime]=(datala.count(crime)/len(datala))*100

In [25]:
Crimes

,Dc_Dist,Psa,Dispatch_Date_Time,Dispatch_Date,Dispatch_Time,Hour,Dc_Key,Location_Block,UCR_General,Text_General_Code,Police_Districts,Month,Lon,Lat
3,35,D,2009-07-19 01:09:00,2009-07-19,01:09:00,1,200935061008,5500 BLOCK N 5TH ST,1500.0,Weapon Violations,20.0,2009-07,-75.130477,40.036389
4,9,R,2009-06-25 12:14:00,2009-06-25,00:14:00,0,200909030511,1800 BLOCK WYLIE ST,2600.0,All Other Offenses,8.0,2009-06,-75.166350,39.969532
5,17,1,2015-04-25 12:50:00,2015-04-25,12:50:00,12,201517017705,800 BLOCK S BROAD ST,600.0,Thefts,13.0,2015-04,-75.166412,39.940070
6,23,K,2009-02-10 02:33:00,2009-02-10,14:33:00,14,200923006310,2200 BLOCK RIDGE AVE,800.0,Other Assaults,16.0,2009-02,-75.171149,39.979586
12,22,3,2015-10-06 06:18:00,2015-10-06,18:18:00,18,201522089374,1500 BLOCK N 15TH ST,600.0,Thefts,16.0,2015-10,-75.160229,39.976134
13,22,3,2015-10-09 12:49:00,2015-10-09,00:49:00,0,201522090172,1500 BLOCK N 17TH ST,600.0,Thefts,16.0,2015-10,-75.163073,39.977872
15,2,1,2015-11-30 08:39:00,2015-11-30,08:39:00,8,201502076521,6500 BLOCK CASTOR AV,600.0,Thefts,2.0,2015-11,-75.075602,40.041270
18,24,2,2015-04-11 05:38:00,2015-04-11,17:38:00,17,201524030846,2800 BLOCK RUTH ST,2400.0,Disorderly Conduct,17.0,2015-04,-75.122057,39.990907
21,22,4,2015-12-10 12:29:00,2015-12-10,12:29:00,12,201522110572,2600 BLOCK W THOMPSON ST,600.0,Thefts,16.0,2015-12,-75.178874,39.975724
22,15,I,2009-01-11 12:30:00,2009-01-11,12:30:00,12,200915003568,5200 BLOCK HARBISON AVE,600.0,Thefts,11.0,2009-01,-75.069032,40.012430


In [26]:
countspr

{100: 0.15584422891097985,
 200: 0.5250131393938485,
 300: 4.119932542516078,
 400: 4.325943235005502,
 500: 5.3334452311150224,
 600: 19.018997148950795,
 700: 6.337102694501709,
 800: 12.350795794831354,
 900: 0.2528483533444387,
 1000: 0.21941967247394276,
 1100: 5.040135515090025,
 1200: 0.2075109979422373,
 1300: 0.03488210177790885,
 1400: 8.98007828781391,
 1500: 0.8360920981256027,
 1600: 0.5770549847883093,
 1700: 0.6894841246585047,
 1800: 6.188900646490997,
 1900: 0.04191478358797111,
 2000: 0.08092272536111648,
 2100: 2.3794844481618678,
 2200: 0.2505978951652188,
 2300: 0.21004276339385972,
 2400: 1.83759287242387,
 2500: 0.3152047987269908,
 2600: 19.690758915447944}

In [18]:
mapIndex = range(100,2601,100)
countspr = {}
for ind in range(100,2601,100):
    

In [ ]:
newData = []
for item in data:
    date = ''.join( [c for c in item[1] if c not in ' -:' ] )
    year = date[:4]
    month = date[4:6]
    day = date[6:8]
    hour = item[2]
    minute = date[10:12]
    item[2] = int(item[2])
    #lon = float("{0:.4f}".format(item[4]))
    #lat = float("{0:.4f}".format(item[5]))
    lon = item[4]
    lat = item[5]
    newData.append([item[0],int(year),int(month),int(day),int(hour),int(minute),lon,lat,item[3]])
data = np.array(newData)

In [ ]:
norm = normalizer()
dLength = np.shape(data)[0]

trainSize = int(dLength*0.80)

X_train = norm.fit_transform(data[:trainSize,[2,3,4,5,6,7]])
y_train = data[:trainSize,8]
X_test = norm.fit_transform(data[trainSize:,[2,3,4,5,6,7]])
y_test = data[trainSize:,8]
len(y_train)